In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

import pandas
pandas.set_option('display.max_rows', 1000)

from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler

import lsst.geom
import lsst.afw.display as afwDisplay

# Start TAP service

In [ ]:
service = get_tap_service()

## List all the tables

In [ ]:
# results = service.search("SELECT description, table_name FROM TAP_SCHEMA.tables")
# results_tab = results.to_table()
# results_tab

## Which RA/Dec are covered by the test area? Use Object table to find out.

Using our typical central-to-DC2 coordinates as a default search region doesn't work, so just grab a bunch of bright Objects and see where they are.

In [ ]:
results = service.search("SELECT * from TAP_SCHEMA.columns WHERE table_name = 'dp02_test_PREOPS863_00.Object'")
# results = service.search("SELECT * from TAP_SCHEMA.columns WHERE table_name = 'dp02_dc2_catalogs.DiaObject'")

print(results.fieldnames)

In [ ]:
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
                         "WHERE table_name = 'dp02_test_PREOPS863_00.Object'")
# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp02_dc2_catalogs.DiaObject'")

# results.to_table().to_pandas()

In [ ]:
results = service.search("SELECT coord_ra, coord_dec, detect_isPrimary, r_calibFlux "\
                         "FROM dp02_test_PREOPS863_00.Object "\
                         "WHERE detect_isPrimary = 1 AND r_calibFlux > 10000 ", \
                         maxrec=100000)
# results = service.search("SELECT ra, decl "\
#                          "FROM dp02_dc2_catalogs.DiaObject ",\
#                          maxrec=100000)

results_tab = results.to_table()

In [ ]:
print(len(results_tab))
# results_tab

In [ ]:
print(np.max(results_tab['coord_ra'])-np.min(results_tab['coord_ra']), \
      np.max(results_tab['coord_dec'])-np.min(results_tab['coord_dec']))

In [ ]:
plt.plot( results_tab['coord_ra'], results_tab['coord_dec'], 'o', ms=3, alpha=0.3, mew=0 )
plt.show()

So we can use 57.5, -36.5. as the approximate central coordinate for searches when needed.

## DiaObject

In [ ]:
### Get column names

# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp02_test_PREOPS863_00.DiaObject'")
# results.to_table().to_pandas()

In [ ]:
### Find DiaObjects detected in g, r, and i filters, in a small central area.

# results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, rPSFluxMean, "
#                          "scisql_fluxToAbMag(gPSFluxMax/1e32) AS gPSMagMin, "
#                          "scisql_fluxToAbMag(rPSFluxMax/1e32) AS rPSMagMin, "
#                          "scisql_fluxToAbMag(iPSFluxMax/1e32) AS iPSMagMin "
#                          "FROM dp02_test_PREOPS863_00.DiaObject "\
#                          "WHERE scisql_fluxToAbMag(gPSFluxMax/1e32) > 14 "
#                          "AND scisql_fluxToAbMag(gPSFluxMax/1e32) < 24 "
#                          "AND scisql_fluxToAbMag(rPSFluxMax/1e32) > 14 "
#                          "AND scisql_fluxToAbMag(rPSFluxMax/1e32) < 24 "
#                          "AND scisql_fluxToAbMag(iPSFluxMax/1e32) > 14 "
#                          "AND scisql_fluxToAbMag(iPSFluxMax/1e32) < 24 "
#                          "AND CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 0.5)) = 1 ", maxrec=10000)
# DiaObjs = results.to_table()
# del results

In [ ]:
### Retrieve all DiaObjects with an r-band detection in a small central area.

# results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
#                          "scisql_fluxToAbMag(rPSFluxMean/1e32) AS rPSMagMean, "
#                          "scisql_fluxToAbMag(rPSFluxMax/1e32) AS rPSMagMin "
#                          "FROM dp02_test_PREOPS863_00.DiaObject "\
#                          "WHERE scisql_fluxToAbMag(rPSFluxMax/1e32) > 16 "
#                          "AND scisql_fluxToAbMag(rPSFluxMax/1e32) < 25 "
#                          "AND CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 0.5)) = 1 ", maxrec=10000)
# DiaObjs = results.to_table()
# del results

In [ ]:
### Retrieve all DiaObjects in a small central area.

# results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
#                          "scisql_fluxToAbMag(rPSFluxMean/1e32) AS rPSMagMean, "
#                          "scisql_fluxToAbMag(rPSFluxMax/1e32) AS rPSMagMin "
#                          "FROM dp02_test_PREOPS863_00.DiaObject "\
#                          "WHERE CONTAINS(POINT('ICRS', ra, decl), CIRCLE('ICRS', 57.5, -36.5, 0.1)) = 1 ", maxrec=10000)
# DiaObjs = results.to_table()
# del results

In [ ]:
# print(len(DiaObjs))

In [ ]:
# DiaObjs

## DiaSource

In [ ]:
# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp02_test_PREOPS863_00.DiaSource'")
# results.to_table().to_pandas()

## Truth

In [ ]:
### Get truth catalog column names

# results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "\
#                          "WHERE table_name = 'dp01_dc2_catalogs.truth_match'")
# results.to_table().to_pandas()

In [ ]:
### Get all the SNe (truth_type=3) with redshift < 0.4 in a small central area

# query = "SELECT id, ra, dec, match_objectId, truth_type, redshift, mag_r "\
#         "FROM dp01_dc2_catalogs.truth_match "\
#         "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 0.5)) = 1 "\
#         "AND redshift < 0.4 AND truth_type = 3"
# results = service.search(query, maxrec=10000)
# Truth = results.to_table()
# del results

In [ ]:
### Get all the SNe (truth_type=3) in a small central area

query = "SELECT id, ra, dec, match_objectId, truth_type, redshift, mag_r "\
        "FROM dp01_dc2_catalogs.truth_match "\
        "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 0.1)) = 1 "\
        "AND truth_type = 3"
results = service.search(query, maxrec=10000)
Truth = results.to_table()
del results

In [ ]:
print(len(Truth))

In [ ]:
# Truth

In [ ]:
plt.hist(Truth['redshift'], bins=20)
plt.axvline(0.6, color='magenta', label='peak r~24.5 mag')
plt.legend(loc='upper left')
plt.show()

## Crude match of truth SNeIa to DiaObjects

In [ ]:
tx = np.where(Truth['redshift'] < 0.6)[0]

fig = plt.figure(figsize=(10,10))
plt.plot(DiaObjs['ra'], DiaObjs['decl'], 'o', ms=10, alpha=0.5, mew=1, color='None', markeredgecolor='dodgerblue', label='DiaObject')
plt.plot(Truth['ra'][tx], Truth['dec'][tx], '*', ms=7, alpha=1, color='black', label='Truth SNIa, z<0.6')
plt.legend(loc='lower left', fontsize=16)
plt.title('Why do so many SNIa not become DiaObjects?', fontsize=18)
plt.show()

In [ ]:
index_truth = []
index_diaobj = []

cosdec = np.cos(np.radians(36.6))
for i in range(len(Truth)):
    tx = np.where( (np.absolute(Truth['ra'][i] - DiaObjs['ra']) < 3.0/3600.0) & \
                   (np.absolute(Truth['dec'][i] - DiaObjs['decl'])*cosdec < 3.0/3600.0)  )[0]
    if len(tx) == 1:
        index_truth.append(i)
        index_diaobj.append(tx[0])
        offset = 3600.0 * np.sqrt( (Truth['ra'][i] - DiaObjs['ra'][tx[0]])**2 + 
                                   ((Truth['dec'][i] - DiaObjs['decl'][tx[0]])*cosdec)**2 )
        print(i, len(tx), ' match: ', Truth['id'][i], np.round(Truth['redshift'][i],3), np.round(offset,2), DiaObjs['nDiaSources'][tx[0]])
    del tx

In [ ]:
filter_names = ['u', 'g', 'r', 'i', 'z', 'y']
filter_color = ['darkviolet', 'darkgreen', 'red', 'darkorange', 'brown', 'black']
filter_symbol = ['o', '^', 'v', 's', '*', 'p']

fig, ax = plt.subplots(len(index_truth), figsize=(14,14), sharey=False, sharex=False)

for a, (i, j) in enumerate(zip(index_truth,index_diaobj)):
    results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, "
                             "filterName, midPointTai, "
                             "scisql_fluxToAbMag(psFlux/1e32) AS psAbMag "
                             "FROM dp02_test_PREOPS863_00.DiaSource "
                             "WHERE diaObjectId = "+str(DiaObjs['diaObjectId'][j]))
    results = results.to_table()
    print(len(results))
    print(results)

    for f, filt in enumerate(filter_names):
        fx = np.where(results['filterName'] == filt)[0]
        ax[a].plot(results['midPointTai'][fx], results['psAbMag'][fx], 
                   filter_symbol[f], ms=15, mew=0, alpha=0.5, color=filter_color[f])
        del fx
    
    # ax[a].set_ylim([np.max(results['psAbMag'])+0.3, np.min(results['psAbMag'])-0.3])
    
    del results

### How many low-z SNeIa are there, in total?


In [ ]:
t1 = time.time()

query = "SELECT id, ra, dec, match_objectId, match_sep, truth_type, redshift, mag_r "\
        "FROM dp01_dc2_catalogs.truth_match "\
        "WHERE redshift < 0.2 AND redshift > 0.01 AND truth_type = 3"
results = service.search(query, maxrec=10000)

t2 = time.time()
print('Time elapsed: ',t2-t1,' seconds')
print(len(results))

TrueSNIa = results.to_table()
del results

In [ ]:
plt.hist( TrueSNIa['redshift'] )
plt.show()

tx = np.where( TrueSNIa['redshift'] < 0.1 )[0]
print('Number with redshift < 0.1 ', len(tx))